# 🔬 ViTHSD - Dataset A - Qwen2.5-3B (QLoRA)

**Model 3/3 cho Dataset A**

- **Model**: Qwen2.5-3B-Instruct (4-bit quantization)
- **Dataset**: A (Original ViTHSD - chỉ content + labels)
- **Task**: Multi-Label Classification (11 labels)
- **Technique**: QLoRA (4-bit + LoRA fine-tuning)

### 📌 Label Constraint
- Each category can only have **ONE level**: offensive OR hate (not both)

## Step 1: Setup Environment

In [1]:
# Cài đặt các thư viện cần thiết
!pip uninstall -y protobuf
!pip install -q protobuf==3.20.0
!pip install -q transformers datasets accelerate sentencepiece
!pip install -q scikit-learn pandas numpy tqdm openpyxl

# Install bitsandbytes - QUAN TRỌNG: dùng -U để lấy latest version
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes
!pip install -q peft

# Suppress warnings
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'
warnings.filterwarnings('ignore')

print("✅ Dependencies installed!")

Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.0 which is incompatible.
google-cloud-secret-manager 2.25.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-vision 3.11.0 requires protobuf!=4.21.0,!=4.

In [2]:
# 🎮 GPU Auto-Detection & Optimization
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"\n{'='*70}")
    print(f"🎮 DETECTED: {num_gpus} GPU(s)")
    print(f"{'='*70}")
    
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
    
    # Auto-configure based on number of GPUs
    if num_gpus == 1:
        # Single GPU (P100)
        print(f"\n{'='*70}")
        print("⚙️  SINGLE GPU MODE (P100 Optimized)")
        print(f"{'='*70}")
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        
        BATCH_SIZE_QWEN = 2  # P100 (16GB)
        
        GPU_CONFIG = {
            'num_gpus': 1,
            'device': 'cuda:0',
            'batch_sizes': {
                'qwen': BATCH_SIZE_QWEN
            },
            'gradient_accumulation_steps': 2
        }
        
    elif num_gpus >= 2:
        # Multi-GPU (T4 x2)
        print(f"\n{'='*70}")
        print("⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)")
        print(f"{'='*70}")
        
        BATCH_SIZE_QWEN = 4  # T4 x2 (32GB)
        
        GPU_CONFIG = {
            'num_gpus': num_gpus,
            'device': 'cuda',
            'batch_sizes': {
                'qwen': BATCH_SIZE_QWEN
            },
            'gradient_accumulation_steps': 1
        }
    
    print(f"\n📊 Configured Batch Size:")
    print(f"   Qwen:   {GPU_CONFIG['batch_sizes']['qwen']}")
    print(f"\n{'='*70}\n")
    
else:
    print("❌ No GPU detected! This notebook requires GPU.")
    GPU_CONFIG = None

PyTorch version: 2.6.0+cu124
CUDA available: True

🎮 DETECTED: 2 GPU(s)
GPU 0: Tesla T4
GPU 1: Tesla T4

⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)

📊 Configured Batch Size:
   Qwen:   4




## Step 2: Import và Setup Paths

In [3]:
import sys
import os

# Copy files ra working directory
!cp -r /kaggle/input/vithsd-experiment/* /kaggle/working/

os.chdir("/kaggle/working")
sys.path.insert(0, "/kaggle/working")

print("📁 Files copied to working directory")

📁 Files copied to working directory


In [4]:
# Import modules
from config import FINAL_LABELS, DATA_DIR, OUTPUT_DIR
from data_preparation import load_dataset_A
from evaluation import Evaluator
from models import create_model

print("✅ All modules imported!")
print(f"📊 Labels: {len(FINAL_LABELS)}")

✅ All modules imported!
📊 Labels: 11


## Step 3: Load Dataset A

In [5]:
# Load Dataset A
train_texts, train_labels = load_dataset_A('train')
test_texts, test_labels = load_dataset_A('test')

print(f"📊 Dataset A:")
print(f"   Train: {len(train_texts)} samples")
print(f"   Test: {len(test_texts)} samples")

[ViHSDLoader] Loaded 7000 samples from train
[ViHSDLoader] Loaded 1800 samples from test
📊 Dataset A:
   Train: 7000 samples
   Test: 1800 samples


## Step 4: Train Qwen with QLoRA

In [6]:
# Initialize
evaluator = Evaluator()

# Qwen - Dataset A (QLoRA 4-bit)
print("="*70)
print("🚀 QWEN2.5-3B - DATASET A (QLoRA 4-bit)")
print("="*70)

model_qwen = create_model(
    "qwen", 
    dataset_type="A",
    batch_size=GPU_CONFIG['batch_sizes']['qwen'],  # Auto-adjusted
    num_epochs=2,
    learning_rate=2e-4,
    lora_r=8,
    lora_alpha=16
)

# Train
model_qwen.train(train_texts, train_labels)

# Predict
pred_qwen, _ = model_qwen.predict(test_texts)

# Evaluate
result = evaluator.evaluate(test_labels, pred_qwen, "Qwen", "A")
evaluator.print_result(result)


🚀 QWEN2.5-3B - DATASET A (QLoRA 4-bit)


E0000 00:00:1765782060.172321      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765782060.220759      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[Qwen2.5-3B_MultiLabel] Training with QLoRA (4-bit)...
  Model: Qwen/Qwen2.5-3B-Instruct
  Device: cuda
  Original samples: 7000
  Mode: Standard (Dataset A)
  [Oversampling] Added 540 samples for minority labels
  LoRA config: r=8, alpha=16


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

  Epoch 1: avg_loss = 0.2483


Epoch 2/2: 100%|██████████| 1885/1885 [2:15:35<00:00,  4.32s/it, loss=0.1012]


  Epoch 2: avg_loss = 0.1896
  ✓ Training completed


Predicting: 100%|██████████| 1800/1800 [29:11<00:00,  1.03it/s]


📊 Qwen on Dataset A (Multi-Label)
  Subset Accuracy (Exact Match): 0.5578
  Hamming Loss:                  0.0911
  F1 Samples:                    0.6332
  F1 Macro:                      0.3310
  F1 Micro:                      0.5900
  Precision (Samples):           0.6469
  Recall (Samples):              0.6372

  F1 per Label:
    normal                   : 0.7970
    individuals#offensive    : 0.4337
    individuals#hate         : 0.6190
    groups#offensive         : 0.2419
    groups#hate              : 0.4583
    race#offensive           : 0.2667
    race#hate                : 0.3288
    politics#offensive       : 0.1299
    politics#hate            : 0.3659


## Step 5: Save Results

In [7]:
import json
from pathlib import Path
import pandas as pd
import numpy as np

# Create output directory
output_dir = Path("/kaggle/working/outputs")
output_dir.mkdir(exist_ok=True)

# Convert numpy types
def convert_to_serializable(obj):
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Save
results = {'Qwen': convert_to_serializable(result)}
with open(output_dir / "results_datasetA_qwen.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Results saved to /kaggle/working/outputs/results_datasetA_qwen.json")
print(f"\n📊 Qwen Results:")
print(f"   Subset Accuracy: {result['subset_accuracy']:.4f}")
print(f"   Macro F1: {result['f1_macro']:.4f}")
print(f"   Micro F1: {result['f1_micro']:.4f}")

✅ Results saved to /kaggle/working/outputs/results_datasetA_qwen.json

📊 Qwen Results:
   Subset Accuracy: 0.5578
   Macro F1: 0.3310
   Micro F1: 0.5900


## Step 6: Save Model State Dict

Lưu LoRA adapters và base model state để có thể load lại model sau này

In [8]:
"""
FIX for dataseta-qwen.ipynb - Cell Save Model Checkpoint

Thay thế cell cuối cùng (Step 6: Save Model State Dict) bằng code này:
"""

import torch
from pathlib import Path

# Create model checkpoint directory
checkpoint_dir = Path("/kaggle/working/checkpoints")
checkpoint_dir.mkdir(exist_ok=True)

# Save LoRA adapters (nhẹ hơn, khuyến nghị cho Qwen)
lora_adapter_dir = checkpoint_dir / "qwen_datasetA_lora_adapters"
model_qwen.model.save_pretrained(lora_adapter_dir)  # ← FIX: model → model_qwen

# Save tokenizer
tokenizer_dir = checkpoint_dir / "qwen_datasetA_tokenizer"
model_qwen.tokenizer.save_pretrained(tokenizer_dir)  # ← FIX: model → model_qwen

# Save additional metadata
metadata_path = checkpoint_dir / "qwen_datasetA_metadata.pt"
torch.save({
    'model_config': {
        'base_model': 'Qwen/Qwen2.5-3B-Instruct',
        'quantization': '4-bit',
        'lora_config': {
            'r': 8,  # ← FIX: Updated to match actual config
            'lora_alpha': 16,  # ← FIX: Updated to match actual config
            'target_modules': ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']  # ← FIX: Complete list
        },
        'labels': model_qwen.labels  # ← FIX: model → model_qwen
    },
    'training_info': {
        'dataset': 'A',
        'technique': 'QLoRA (4-bit + LoRA)',
        'accuracy': result['subset_accuracy'],
        'macro_f1': result['f1_macro'],
        'micro_f1': result['f1_micro']
    }
}, metadata_path)

print(f"✅ LoRA adapters saved to: {lora_adapter_dir}")
print(f"✅ Tokenizer saved to: {tokenizer_dir}")
print(f"✅ Metadata saved to: {metadata_path}")

# Calculate total size
total_size = sum(f.stat().st_size for f in lora_adapter_dir.rglob('*') if f.is_file())
print(f"📦 LoRA adapters size: {total_size / (1024**2):.2f} MB")

print("\n💡 Cách load lại:")
print("   from peft import PeftModel")
print("   base_model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-3B-Instruct', ...)")
print("   model = PeftModel.from_pretrained(base_model, 'qwen_datasetA_lora_adapters')")
print("   tokenizer = AutoTokenizer.from_pretrained('qwen_datasetA_tokenizer')")
print("   # Sau đó có thể dùng model.predict(..., extract_hate_words=True)")


✅ LoRA adapters saved to: /kaggle/working/checkpoints/qwen_datasetA_lora_adapters
✅ Tokenizer saved to: /kaggle/working/checkpoints/qwen_datasetA_tokenizer
✅ Metadata saved to: /kaggle/working/checkpoints/qwen_datasetA_metadata.pt
📦 LoRA adapters size: 57.16 MB

💡 Cách load lại:
   from peft import PeftModel
   base_model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-3B-Instruct', ...)
   model = PeftModel.from_pretrained(base_model, 'qwen_datasetA_lora_adapters')
   tokenizer = AutoTokenizer.from_pretrained('qwen_datasetA_tokenizer')
   # Sau đó có thể dùng model.predict(..., extract_hate_words=True)
